In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Upload the csv file containing the translated train data from google drive and do some processessing on that csv file**
1 - delete all Non entries in both coulmns context and translated_utterance
2 - crop only 3 coulmns context , translated_utterance "sentence", and translated_utterance "response" then put these in a new dataframe.

In [8]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/nlp/train_arabic.csv" , encoding='utf-16')
data = data[data['translated_utterance'].notna()]
data = data[data['context'].notna()]
data.reset_index(drop=True , inplace = True)
column_names = [ "context","emotion", "response"]
trained_data = pd.DataFrame(columns = column_names)
i = 0
j = 0
while(i < data.shape[0]-1):#data.shape[0] ):
  first = data.loc[i]
  second = data.loc[i+1]
  if(first['context']== second['context']):
    trained_data.loc[j] = [first['translated_utterance'],first['context'], second['translated_utterance']]
    i = i+2
    j = j+1
  else:
    i = i+1
  

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


KeyboardInterrupt: ignored

In [ ]:
# import requests
# file_url = "https://github.com/aub-mind/arabert"
	
# r = requests.get(file_url, stream = True)

# with open("/content/drive/MyDrive/nlp/arabert", "wb") as file:
# 	for block in r.iter_content(chunk_size = 1024):
# 		if block:
# 			file.write(block)
# !git clone https://github.com/aub-mind/arabert 
# import sys
# sys.path.append('/content/gdrive/MyDrive')

# # Now we can import the library and use the function.

# import mylib
# mylib.MyFunction()

In [ ]:
trained_data.to_csv( "/content/drive/MyDrive/nlp/train_crop_arabic.csv", encoding='utf-8' , index=False)

# import a model from github that preprocess on the data "https://github.com/aub-mind/arabert"
import also the required libraries for the preprocess model.

In [9]:
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic
!pip install farasapy
!pip install datasets
import pandas as pd
from datasets import load_dataset 
from arabert.preprocess import ArabertPreprocessor
model_name="bert-base-arabert"
arabert_prep = ArabertPreprocessor(model_name=model_name, keep_emojis=False)

fatal: destination path 'arabert' already exists and is not an empty directory.


[2021-06-23 20:55:56,888 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


run the preprocess model on the data to carry out the pipeline for he data preprocessing (tokenization , lemmatization , normalization) with the emotion and context coulmns.

In [10]:
text = "ولقد كانوا جميعا موجودين فى القاعة "
arabert_prep.preprocess(text)

'و+ لقد كان +وا جميع +ا موجود +ين في ال+ قاع +ة'

In [11]:
for index,row in trained_data.iterrows():
  row['context'] = arabert_prep.preprocess(row['context'])
  row['emotion'] = arabert_prep.preprocess(row['emotion'])                

In [ ]:
trained_data.to_csv( "/content/drive/MyDrive/nlp/train_edited_arabic.csv", encoding='utf-8' , index=False)

**load the preprocessed data in the form of data set to be convenient input to the train model.
give all the training data a tag 'train'.**

In [12]:
train_data = load_dataset('csv', data_files="/content/drive/MyDrive/nlp/train_edited_arabic.csv")
train_data = train_data['train']

Using custom data configuration default-d0a74f89d531f3a7


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d0a74f89d531f3a7/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [13]:
train_data
#train_data

Dataset({
    features: ['context', 'emotion', 'response'],
    num_rows: 37682
})

# **Testing**

**Upload the csv file containing the translated test data from google drive and do some processessing on that csv file
1 - delete all Non entries in both coulmns context and translated_utterance.
2 - crop only 3 coulmns context , translated_utterance "sentence", and translated_utterance "response" then put these in a dataframe.**

In [14]:
test_data=pd.read_csv("/content/drive/MyDrive/nlp/test_arabic.csv" , encoding='utf-16')
test_data = test_data[test_data['translated_utterance'].notna()]
test_data = test_data[test_data['context'].notna()]
test_data.reset_index(drop=True , inplace = True)
column_names = [ "context","emotion", "response"]
tested_data = pd.DataFrame(columns = column_names)
i = 0
j = 0
while(i < test_data.shape[0]-1):#data.shape[0] ):
  first = test_data.loc[i]
  second = test_data.loc[i+1]
  if(first['context']== second['context']):
    tested_data.loc[j] = [ first['translated_utterance'],first['context'], second['translated_utterance']]
    i = i+2
    j = j+1
  else:
    i = i+1


In [15]:
tested_data.isna().any()

context     False
emotion     False
response    False
dtype: bool

In [8]:
tested_data.to_csv( "/content/drive/MyDrive/nlp/test_crop_arabic.csv", encoding='utf-8' , index=False)

In [16]:
for index,row in tested_data.iterrows():
  row['context'] = arabert_prep.preprocess(row['context'])
  row['emotion'] = arabert_prep.preprocess(row['emotion'])

[2021-06-23 20:59:19,422 - farasapy_logger - ERROR]: pipe broke! error code and message: [[Errno 32] Broken pipe]. reinitailize the process.., This may take sometime depending on the running task


In [17]:
tested_data.to_csv( "/content/drive/MyDrive/nlp/test_edited_arabic.csv", encoding='utf-8' , index=False)

**load the preprocessed test data in the form of dataset to fit the input to the train model.
Here we divide the preprocessed test data randomly between the dev data and the test data**

In [18]:
test_dataset = load_dataset('csv', data_files="/content/drive/MyDrive/nlp/test_edited_arabic.csv")
test_dataset = test_dataset['train']
dev_data = test_dataset.train_test_split(test_size=0.5,seed=42)['train']
test_data = test_dataset.train_test_split(test_size=0.5,seed=42)['test']

Using custom data configuration default-db44218a061e073b


Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-db44218a061e073b/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-5999f95310988da9.arrow and /root/.cache/huggingface/datasets/csv/default-db44218a061e073b/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-e5fc55fc82168ae7.arrow


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-db44218a061e073b/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [19]:
test_data

Dataset({
    features: ['context', 'emotion', 'response'],
    num_rows: 2624
})

In [20]:
print(len(test_data))
print(len(dev_data))

2624
2624


# **Model**

# import the chatbot model from github
**impot also the required libraries to run the model**

In [21]:
!pip install git-python==1.0.3
!pip install sacrebleu==1.4.2
!pip install rouge_score
!pip install -U transformers==4.5.1
!git clone https://github.com/aub-mind/Arabic-Empathetic-Chatbot
import os
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sacrebleu import corpus_bleu

     |████████████████████████████████| 174kB 13.6MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-cp37-none-any.whl size=26323 sha256=e86b5a5c10a18f97ec22be2ddb9eb7af4c62bf628ae74505cda620628bb59ba8
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built typing


     |████████████████████████████████| 2.1MB 13.8MB/s 
     |████████████████████████████████| 3.3MB 80.5MB/s 
     |████████████████████████████████| 901kB 48.6MB/s 
Cloning into 'Arabic-Empathetic-Chatbot'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 27 (delta 5), reused 14 (delta 2), pack-reused 0
Unpacking objects: 100% (27/27), done.


**set the configurations and the parameters variables to the chatbot model**

In [22]:
encoder_max_length=150
decoder_max_length=150
model_name = "aubmindlab/bert-base-arabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
batch_size=4
def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["context"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["response"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [                                                                                 
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

**map the processed input data [train , dev , test] to the format required to fit inside the chatbot model**

In [24]:
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

dev_data = dev_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
dev_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

test_data = test_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
test_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [25]:
arabert2arabert = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name, tie_encoder_decoder=False)

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabert and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

In [26]:
#set special tokens
arabert2arabert.config.decoder_start_token_id = tokenizer.cls_token_id                                             
arabert2arabert.config.eos_token_id = tokenizer.sep_token_id
arabert2arabert.config.pad_token_id = tokenizer.pad_token_id

#sensible parameters for beam search
#set decoding params                               
arabert2arabert.config.max_length = 64
arabert2arabert.config.early_stopping = True

arabert2arabert.config.num_beams = 1
arabert2arabert.config.vocab_size = arabert2arabert.config.encoder.vocab_size

# import libiraries and dependicies to set up the environment or training 

In [27]:
import torch
import torch.nn as nn

def compute_metrics(pred):
  labels_ids = pred.label_ids
  #pred_ids = torch.argmax(pred.predictions,dim=2)
  pred_ids = pred.predictions  

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  return {"bleu": round(corpus_bleu(pred_str , [label_str]).score, 4)}

In [28]:
#Set training arguments 
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    gradient_accumulation_steps = 2,
    predict_with_generate=True,
    do_eval=True,
    evaluation_strategy ="epoch",
    do_train=True,
    logging_steps=500,  
    save_steps= 37682 // ( batch_size * 2),  
    warmup_steps=100,
    eval_steps=10,
    #max_steps=16, # delete for full training
    num_train_epochs=3,# uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=0,
    #fp16=True, 
)

**from the libirary transformers we will import a 'Seq2SeqTrainer' trainer to process with the training and dev batches**

In [29]:
trainer = Seq2SeqTrainer(
    model=arabert2arabert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=tokenizer
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()

In [ ]:
trainer._save("./arabert2arabert")
tokenizer.save_pretrained("/content/drive/MyDrive/nlp")

# **load the chatbot and try to chat with it**

In [ ]:
from transformers import pipeline

chatbot = pipeline("text2text-generation",model='"/content/drive/MyDrive/nlp"')

In [ ]:
chatbot( "اريد ان انام",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

# link for the model
https://drive.google.com/file/d/1K1ROGrLWwNfQVmb8NK0EvXyfmVPsekpF/view?usp=sharing
# Team Members
1 - Mohamed Orabi 4020454

2 - Abdelrahman Khaled  40-15358

3 - Ahmed Hisham  40-1893

4 - Hesham Mohamed 40-671

5 - Ahmed Mohamed Salah 40-1316
